In [1]:
import pandas as pd

### Load data

In [2]:
memcache = pd.read_csv('memcache.csv', index_col=0)
memcache.head()

,rdma,tcp,nop
0,1 Threads\n1 Connections per t...,1 Threads\n1 Connections per t...,1 Threads\n1 Connections per t...
1,1 Threads\n1 Connections per t...,1 Threads\n1 Connections per t...,1 Threads\n1 Connections per t...
2,1 Threads\n1 Connections per t...,1 Threads\n1 Connections per t...,1 Threads\n1 Connections per t...
3,1 Threads\n1 Connections per t...,1 Threads\n1 Connections per t...,1 Threads\n1 Connections per t...
4,1 Threads\n1 Connections per t...,1 Threads\n1 Connections per t...,1 Threads\n1 Connections per t...


### Helper

In [28]:
def extract(entry):
    res = {}
    cats = ['Sets', 'Gets', 'Totals']
    cats_index = {'Sets':9, 'Gets':10, 'Totals':12}
    for cat in cats:
        res[cat] = {}
        res[cat]['Ops/sec'] = float(entry.split('\n')[cats_index[cat]].split()[1])
        res[cat]['Avg. Latency'] = float(entry.split('\n')[cats_index[cat]].split()[4])
        res[cat]['p50 Latency'] = float(entry.split('\n')[cats_index[cat]].split()[5])
        res[cat]['p99 Latency'] = float(entry.split('\n')[cats_index[cat]].split()[6])
        res[cat]['p99.9 Latency'] = float(entry.split('\n')[cats_index[cat]].split()[7])
        res[cat]['KB/sec'] = float(entry.split('\n')[cats_index[cat]].split()[8])
    return res

def init():
    res = {}
    cats = ['Sets', 'Gets', 'Totals']
    for cat in cats:
        res[cat] = {}
        res[cat]['Ops/sec'] = 0
        res[cat]['Avg. Latency'] = 0
        res[cat]['p50 Latency'] = 0
        res[cat]['p99 Latency'] = 0
        res[cat]['p99.9 Latency'] = 0
        res[cat]['KB/sec'] = 0
    return res

def add(res1, res2):
    res = {}
    cats = ['Sets', 'Gets', 'Totals']
    for cat in cats:
        res[cat] = {}
        res[cat]['Ops/sec'] = res1[cat]['Ops/sec'] + res2[cat]['Ops/sec']
        res[cat]['Avg. Latency'] = res1[cat]['Avg. Latency'] + res2[cat]['Avg. Latency']
        res[cat]['p50 Latency'] = res1[cat]['p50 Latency'] + res2[cat]['p50 Latency']
        res[cat]['p99 Latency'] = res1[cat]['p99 Latency'] + res2[cat]['p99 Latency']
        res[cat]['p99.9 Latency'] = res1[cat]['p99.9 Latency'] + res2[cat]['p99.9 Latency']
        res[cat]['KB/sec'] = res1[cat]['KB/sec'] + res2[cat]['KB/sec']
    return res

def scale(res, n):
    cats = ['Sets', 'Gets', 'Totals']
    for cat in cats:        
        res[cat]['Ops/sec'] /= n
        res[cat]['Avg. Latency'] /= n
        res[cat]['p50 Latency'] /= n
        res[cat]['p99 Latency'] /= n
        res[cat]['p99.9 Latency'] /= n
        res[cat]['KB/sec'] /= n    

entry = memcache['tcp'][0]
extract(entry)


{'Sets': {'Ops/sec': 138.19,
  'Avg. Latency': 0.66273,
  'p50 Latency': 0.671,
  'p99 Latency': 0.831,
  'p99.9 Latency': 0.871,
  'KB/sec': 9.43},
 'Gets': {'Ops/sec': 1380.36,
  'Avg. Latency': 0.65372,
  'p50 Latency': 0.655,
  'p99 Latency': 0.807,
  'p99.9 Latency': 0.831,
  'KB/sec': 34.9},
 'Totals': {'Ops/sec': 1518.55,
  'Avg. Latency': 0.65454,
  'p50 Latency': 0.655,
  'p99 Latency': 0.807,
  'p99.9 Latency': 0.847,
  'KB/sec': 44.33}}

### Get

In [30]:
res = {}
cases = ['tcp', 'nop', 'rdma']
for case in cases:
    res[case] = init()
    for entry in memcache[case]:
        res[case] = add(res[case], extract(entry))
    scale(res[case], 5)
res
    

{'tcp': {'Sets': {'Ops/sec': 125.56199999999998,
   'Avg. Latency': 0.735676,
   'p50 Latency': 0.743,
   'p99 Latency': 0.9030000000000001,
   'p99.9 Latency': 0.9398,
   'KB/sec': 8.568},
  'Gets': {'Ops/sec': 1254.216,
   'Avg. Latency': 0.7215680000000001,
   'p50 Latency': 0.7285999999999999,
   'p99 Latency': 0.8806,
   'p99.9 Latency': 0.9141999999999999,
   'KB/sec': 31.71},
  'Totals': {'Ops/sec': 1379.776,
   'Avg. Latency': 0.7228519999999999,
   'p50 Latency': 0.7318,
   'p99 Latency': 0.8821999999999999,
   'p99.9 Latency': 0.9222000000000001,
   'KB/sec': 40.278}},
 'nop': {'Sets': {'Ops/sec': 299.624,
   'Avg. Latency': 0.306242,
   'p50 Latency': 0.3046,
   'p99 Latency': 0.41179999999999994,
   'p99.9 Latency': 0.4326,
   'KB/sec': 20.444},
  'Gets': {'Ops/sec': 2992.942,
   'Avg. Latency': 0.29986199999999996,
   'p50 Latency': 0.3014,
   'p99 Latency': 0.40540000000000004,
   'p99.9 Latency': 0.42619999999999997,
   'KB/sec': 75.668},
  'Totals': {'Ops/sec': 3292.568